In [111]:
# import libraries
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from time import sleep

GET THE MAIN SOUP

In [2]:
main_url = 'https://www.baseball-reference.com'
page = requests.get(main_url)
main_soup = BeautifulSoup(page.content)

CRAWL TO THE STANDINGS PAGE AND GET THE SOUP

In [3]:
for this_soup in main_soup.find_all('a'):

    if this_soup.get_text() == "Standings":
        standings_url = this_soup.get("href")
        print(standings_url)
        break

page = requests.get(main_url + standings_url)
standings_soup = BeautifulSoup(page.content)

/leagues/MLB-standings.shtml


CRAWL TO THE LEAGUES PAGE AND GET THE SOUP

In [4]:
for this_soup in main_soup.find_all('a'):
    if this_soup.get_text() == "Seasons":
        leagues_url = this_soup.get("href")
        print(leagues_url)
        break

page = requests.get(main_url + leagues_url)
leagues_soup = BeautifulSoup(page.content)

/leagues/


GRAB THE WINNERS TABLE FROM THE LEAGUES PAGE

In [5]:
winners_table = pd.read_html(main_url + leagues_url, match="Active Major Leagues")
winners_df = winners_table[0]
print(winners_df.head())

     Year  Lg               Champion               MVP
0  2023.0  NL                    NaN               NaN
1     NaN  AL                    NaN               NaN
2  2022.0  NL  Philadelphia Phillies  Paul Goldschmidt
3     NaN  AL         Houston Astros       Aaron Judge
4  2021.0  NL         Atlanta Braves      Bryce Harper


GET THE LINKS TO THE STANDINGS EVERY YEAR

In [6]:
all_standings_urls = []
for i in range(23):
    for this_soup in standings_soup.find_all('a', {"class": "button2 prev"}):
        if this_soup.get_text() == "Previous Season":
            all_standings_urls.append(this_soup.get("href"))
            break
    page = requests.get(main_url + all_standings_urls[i])
    standings_soup = BeautifulSoup(page.content)

print(all_standings_urls)


['/leagues/majors/2022-standings.shtml', '/leagues/majors/2021-standings.shtml', '/leagues/majors/2020-standings.shtml', '/leagues/majors/2019-standings.shtml', '/leagues/majors/2018-standings.shtml', '/leagues/majors/2017-standings.shtml', '/leagues/majors/2016-standings.shtml', '/leagues/majors/2015-standings.shtml', '/leagues/majors/2014-standings.shtml', '/leagues/majors/2013-standings.shtml', '/leagues/majors/2012-standings.shtml', '/leagues/majors/2011-standings.shtml', '/leagues/majors/2010-standings.shtml', '/leagues/majors/2009-standings.shtml', '/leagues/majors/2008-standings.shtml', '/leagues/majors/2007-standings.shtml', '/leagues/majors/2006-standings.shtml', '/leagues/majors/2005-standings.shtml', '/leagues/majors/2004-standings.shtml', '/leagues/majors/2003-standings.shtml', '/leagues/majors/2002-standings.shtml', '/leagues/majors/2001-standings.shtml', '/leagues/majors/2000-standings.shtml']


GET THE STANDINGS TABLES INTO DFS 

In [145]:
standings_dfs = []
for current_url in all_standings_urls:
    current_df = pd.read_html(main_url + current_url)
    print(len(current_df))
    standings_dfs.append(current_df[5])

HTTPError: HTTP Error 429: Too Many Requests

GET ALL THE LINKS to the teams stats by year

In [112]:
all_teams_urls = pd.DataFrame()
for current_url in all_standings_urls:
    page = requests.get(main_url + current_url)
    teams_soup = BeautifulSoup(page.content)
    overall = teams_soup.find_all('table')
    for this_table in overall:
        for inner_soup in this_table.find_all('a'):
            temp_ser = {}

            temp_ser["TEAM"] = inner_soup.get_text()

            temp_split= inner_soup.get("href").split('/')
            temp_year = temp_split[3].split('.')
            temp_ser["YEAR"] = temp_year[0]

            temp_ser["URL"] = inner_soup.get("href")

            new_row = pd.Series(temp_ser)
            all_teams_urls = pd.concat([all_teams_urls, new_row.to_frame().T], ignore_index=True)
    sleep(30)

print(all_teams_urls.head(35))
print(all_teams_urls.tail(35))

                     TEAM  YEAR                    URL
0        New York Yankees  2022  /teams/NYY/2022.shtml
1       Toronto Blue Jays  2022  /teams/TOR/2022.shtml
2          Tampa Bay Rays  2022  /teams/TBR/2022.shtml
3       Baltimore Orioles  2022  /teams/BAL/2022.shtml
4          Boston Red Sox  2022  /teams/BOS/2022.shtml
5     Cleveland Guardians  2022  /teams/CLE/2022.shtml
6       Chicago White Sox  2022  /teams/CHW/2022.shtml
7         Minnesota Twins  2022  /teams/MIN/2022.shtml
8          Detroit Tigers  2022  /teams/DET/2022.shtml
9      Kansas City Royals  2022  /teams/KCR/2022.shtml
10         Houston Astros  2022  /teams/HOU/2022.shtml
11       Seattle Mariners  2022  /teams/SEA/2022.shtml
12     Los Angeles Angels  2022  /teams/LAA/2022.shtml
13          Texas Rangers  2022  /teams/TEX/2022.shtml
14      Oakland Athletics  2022  /teams/OAK/2022.shtml
15         Atlanta Braves  2022  /teams/ATL/2022.shtml
16          New York Mets  2022  /teams/NYM/2022.shtml
17  Philad

DECIDE WHICH TEAMS STATS I WANT TO GRAB

In [122]:
these_teams = [["New York Yankees", 2022], ["Boston Red Sox", 2022]]

GET THE SOUP TO THE TEAM STATS BY YEAR, THEN GET THE LINKS TO THE BATTING GAME LOGS AND THE PITCHING GAME LOGS

In [134]:
batting_pitching_logs = []
for team, year in these_teams:
    new = all_teams_urls.loc[(all_teams_urls['YEAR'] == str(year)) & (all_teams_urls['TEAM'] == team), ['URL']]
    current_url = new.iloc[0]["URL"]
    page = requests.get(main_url + current_url)
    year_team_soup = BeautifulSoup(page.content)

    for this_li in year_team_soup.find_all("li", {"class" : "full hasmore"}):
        for this_href in this_li.find_all("a"):
            if this_href.get_text() == "Game Logs":
                batting_pitching_logs.append(this_href.get("href"))


GET THE BATTING AND PITCHING DATA

In [138]:
for current_url in batting_pitching_logs:
    this_df = pd.read_html(main_url + current_url)
    print(this_df[0].head())

  Rk Gtm    Date Unnamed: 3  Opp   Rslt  PA  AB  R   H  ... SB CS    BA   OBP  \
0  1   1   Apr 8        NaN  BOS  W,6-5  46  39  6   9  ...  0  0  .231  .326   
1  2   2   Apr 9        NaN  BOS  W,4-2  30  27  4   4  ...  0  0  .197  .289   
2  3   3  Apr 10        NaN  BOS  L,3-4  41  36  3  11  ...  1  0  .235  .325   
3  4   4  Apr 11        NaN  TOR  L,0-3  35  29  0   4  ...  0  0  .214  .316   
4  5   5  Apr 12        NaN  TOR  W,4-0  33  28  4   7  ...  0  0  .220  .319   

    SLG   OPS LOB   # Thr Opp. Starter (GmeSc)  
0  .487  .813  12  12   R        N.Eovaldi(51)  
1  .439  .729   2  10   R        N.Pivetta(46)  
2  .412  .737  11  11   R          T.Houck(36)  
3  .351  .667   8  10   R         A.Manoah(73)  
4  .377  .696   5  10   L        Y.Kikuchi(40)  

[5 rows x 32 columns]
    Avg   0   1   2   3   4   5   6   7  8  9  \
0  5.06  12  15  25  17  14  19  15  11  5  7   

                                   Other Game Totals  
0  10, 10, 10, 10, 10, 10, 10, 11, 12, 12,

In [141]:
for year in standings_dfs:
    print(year.head(8))

                     Tm    W   L   W-L%    GB
0   Los Angeles Dodgers  111  51  0.685    --
1      San Diego Padres   89  73  0.549  22.0
2  San Francisco Giants   81  81  0.500  30.0
3  Arizona Diamondbacks   74  88  0.457  37.0
4      Colorado Rockies   68  94  0.420  43.0
                     Tm    W    L   W-L%    GB
0  San Francisco Giants  107   55  0.660    --
1   Los Angeles Dodgers  106   56  0.654   1.0
2      San Diego Padres   79   83  0.488  28.0
3      Colorado Rockies   74   87  0.460  32.5
4  Arizona Diamondbacks   52  110  0.321  55.0
                     Tm   W   L   W-L%    GB
0   Los Angeles Dodgers  43  17  0.717    --
1      San Diego Padres  37  23  0.617   6.0
2  San Francisco Giants  29  31  0.483  14.0
3      Colorado Rockies  26  34  0.433  17.0
4  Arizona Diamondbacks  25  35  0.417  18.0
                     Tm    W   L   W-L%    GB
0   Los Angeles Dodgers  106  56  0.654    --
1  Arizona Diamondbacks   85  77  0.525  21.0
2  San Francisco Giants   77  85  